## Visualisation of excess mortality over Europe in 2020

The purpose of this notebook is to dynamically generate maps of weekly excess mortality over Europe NUTS2 or NUTS3 regions, depending on their availability.

Data source: 

* Eurostat data on *"Deaths by age group, sex, week and NUTS 3 region"* [demo_r_mweek3](https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=demo_r_mweek3&lang=en),
* Eurostat geographical data on regional units NUTS 2016 (see [GISCO website](https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts)).

Note: we provide here with some quick and dirty material to fetch the data from the provider (*e.g.*, in a bulk manner, not using the API) and compute basic descriptive statistics (*e.g.*, an excess rate). This will need to be improved in order to further automate it.

### Setting

In [13]:
import requests
import io
import zipfile

### (Dirty) data ingestion

Data on death figures will be fetched from [Eurostat bulk website](https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&dir=data). The link to the data source in TSV format is : https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1&file=data%2Fdemo_r_mweek3.tsv.gz. 

In [30]:
bulk_domain = 'https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?sort=1'
death_data = 'demo_r_mweek3'
ext = 'tsv'
gzip = 'gz'
url = '{}&file=data%2F{}.{}.{}'.format(bulk_domain, data, ext, gzip)

In [34]:
gisco_domain = 'https://ec.europa.eu/eurostat/cache/GISCO/distribution/v2/nuts/download'
nuts_year = 2016
nuts_res = 60
nuts_data = 'ref-nuts-%s-%sm' % (nuts_year, nuts_res) 
nuts_ext = 'geojson'
gzip = 'zip'
nuts_url = '%s/%s.%s.%s' %  (gisco_domain, nuts_data, nuts_ext, gzip)

Fetch the URL to get the response:

In [11]:
try:
    response = requests.get(url)
    response.raise_for_status # check status code of request is 200!
except:
    raise IOError('Ups, something got wrong...')
else:
    print('OK status code: %s' % response.status_code)
    source = response.content

OK status code: 200


Retrieve the data: 

In [14]:
try:
    assert any([url.endswith(ext) for ext in ['gzip', 'gz', 'zip']])
except:
    pass
else:
    source = io.BytesIO(source)
try:
    assert zipfile.is_zipfile(source)
except:
    raise IOError('Ups, corrupted file...')
else:
    with zipfile.ZipFile(source) as zf:
        namelist = zf.namelist()
    print(namelist)

OSError: Ups, corrupted file...

In [15]:
zipfile.is_zipfile(source)

False

In [17]:
any([url.endswith(ext) for ext in ['gzip', 'gz', 'zip']])

True

In [18]:
source

In [22]:
with zipfile.ZipFile(source.read()) as zf:
    namelist = zf.namelist()

AttributeError: 'bytes' object has no attribute 'seek'

In [27]:
source = response.content
source = io.BytesIO(source)

In [28]:
x=source.read()

In [29]:
x[:10]

b'\x1f\x8b\x08\x085\xf4\xba^\x00\x03'

In [33]:
!wget -O $data.$ext $source
! unzip $data.$ext
ext = ext.split('.')[0]

/bin/sh: wget: command not found


In [ ]:
!wget -O NUTS2.geojson
!wget -O NUTS3.geojson 